In [1]:
import numpy as np
from qiskit import *
%matplotlib inline
from math import pi, sqrt
import random
import time

from qiskit.visualization import plot_histogram
from qiskit.visualization import array_to_latex
from qiskit_textbook.tools import random_state

from qiskit_nature.drivers import PySCFDriver, UnitsType #Python Chemistry Library
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem #2'nd Quantization

#pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

#pip install qiskit-nature
#pip install pyscf
import warnings
warnings.filterwarnings('ignore')



In [ ]:
"""
 /$$$$$$$$ /$$$$$$$   /$$$$$$  /$$      /$$                    /$$$$$$  /$$$$$$  /$$$$$$  /$$   /$$ /$$$$$$ /$$$$$$$$
| $$_____/| $$__  $$ /$$__  $$| $$$    /$$$                   /$$__  $$|_  $$_/ /$$__  $$| $$  /$$/|_  $$_/|__  $$__/
| $$      | $$  \ $$| $$  \ $$| $$$$  /$$$$                  | $$  \ $$  | $$  | $$  \__/| $$ /$$/   | $$     | $$   
| $$$$$   | $$$$$$$/| $$  | $$| $$ $$/$$ $$                  | $$  | $$  | $$  |  $$$$$$ | $$$$$/    | $$     | $$   
| $$__/   | $$__  $$| $$  | $$| $$  $$$| $$                  | $$  | $$  | $$   \____  $$| $$  $$    | $$     | $$   
| $$      | $$  \ $$| $$  | $$| $$\  $ | $$                  | $$/$$ $$  | $$   /$$  \ $$| $$\  $$   | $$     | $$   
| $$      | $$  | $$|  $$$$$$/| $$ \/  | $$                  |  $$$$$$/ /$$$$$$|  $$$$$$/| $$ \  $$ /$$$$$$   | $$   
|__/      |__/  |__/ \______/ |__/     |__/                   \____ $$$|______/ \______/ |__/  \__/|______/   |__/   
                                                                   \__/                                                                                                                                                                 
"""

In [2]:
# Use PySCF, a classical computational chemistry software
# package, to compute the one-body and two-body integrals in
# electronic-orbital basis, necessary to form the Fermionic operator
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735',
                     unit=UnitsType.ANGSTROM,
                     basis='sto3g')
problem = ElectronicStructureProblem(driver) #Fermionic Operator


# generate the second-quantized operators
second_q_ops = problem.second_q_ops()
main_op = second_q_ops[0]

num_particles = (problem.molecule_data_transformed.num_alpha,
                 problem.molecule_data_transformed.num_beta)

num_spin_orbitals = 2 * problem.molecule_data.num_molecular_orbitals

# setup the classical optimizer for VQE
from qiskit.algorithms.optimizers import L_BFGS_B
optimizer = L_BFGS_B()

# setup the mapper and qubit converter
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization import QubitConverter
mapper = ParityMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)
# map to qubit operators
qubit_op = converter.convert(main_op, num_particles=num_particles)

#Convert Fermionic Operators to Pauli Matrices -> To Circuit

In [3]:
# setup the initial state for the ansatz
from qiskit_nature.circuit.library import HartreeFock #Computational Chemistry Algorithm 
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

# setup the ansatz for VQE
from qiskit.circuit.library import TwoLocal
ansatz = TwoLocal(num_spin_orbitals, ['ry', 'rz'], 'cz')

# add the initial state
ansatz.compose(init_state, front=True)

# set the backend for the quantum computation
backend = Aer.get_backend('aer_simulator_statevector')

# setup and run VQE
from qiskit.algorithms import VQE
algorithm = VQE(ansatz,optimizer=optimizer,quantum_instance=backend)

In [4]:
result = algorithm.compute_minimum_eigenvalue(qubit_op)
print(result.eigenvalue.real)

electronic_structure_result = problem.interpret(result)
print(electronic_structure_result)

-0.965935001118535
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -0.965935001119
  - computed part:      -0.965935001119
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -0.24596600667
 
=== MEASURED OBSERVABLES ===
 
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
